# P2
En ppio voy a tomar el calculo de error de la practica anterior usando MRSE

In [1]:
# Función para calcular RMSE
import numpy as np

def calculate_rmse(reconstructed_image, original_image):
    mean_original = np.mean(original_image)
    mean_reconstructed = np.mean(reconstructed_image)
    image_reconstructed_normalized = reconstructed_image * (mean_original / mean_reconstructed)
    error = image_reconstructed_normalized - original_image
    rmse = np.sqrt(np.mean(error**2))
    return rmse

## Acá defino el ART

In [2]:
import numpy as np
import matplotlib.pyplot as plt


def ART(A, AT, b, x, mu=1e0, niter=1e2, bpos=True):

    ATA = AT(A(np.ones_like(x)))

    for i in range(int(niter)):

        x = x + np.divide(mu * AT(b - A(x)), ATA)

        if bpos:
            x[x < 0] = 0

        # plt.imshow(x, cmap='gray')
        # plt.title("%d / %d" % (i + 1, niter))
        # plt.pause(1)
        # plt.close()

    return x


## Lo que sigue basicamente es la demo sin moficar

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 14 09:06:26 2022

@author: matog
"""
## REFERENCE
# https://en.wikipedia.org/wiki/Algebraic_reconstruction_technique

## ART Equation
# x^(k+1) = x^k + lambda * AT(b - A(x))/ATA

import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import radon, iradon
from scipy.io import loadmat
from scipy.stats import poisson
#from skimage.measure import compare_mse as mse
#from skimage.measure import compare_psnr as psnr
#from skimage.measure import compare_ssim as ssim
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

## SYSTEM SETTING
N = 512
ANG = 180
VIEW = 360
THETA = np.linspace(0, ANG, VIEW + 1)
THETA = THETA[:-1]

A = lambda x: radon(x, THETA, circle=False).astype(np.float32)
AT = lambda y: iradon(y, THETA, circle=False, filter_name=None, output_size=N).astype(np.float32)/(np.pi/(2*len(THETA)))
AINV = lambda y: iradon(y, THETA, circle=False, filter_name='ramp',output_size=N).astype(np.float32)

## DATA GENERATION
x = loadmat('XCAT512.mat')['XCAT512']
p = A(x)
x_full = AINV(p)

## LOW-DOSE SINOGRAM GENERATION
i0 = 1e2
pn = np.exp(-p)
pn = i0*pn
pn = poisson.rvs(pn)
pn[pn <1] = 1
pn = -np.log(pn/i0)
pn[pn < 0] = 0

y = pn

## Algebraic Reconstruction Technique (ART) INITIALIZATION
x_low = AINV(y)
x0 = np.zeros_like(x)
mu = 1e0
niter = 10
bpos = True

x_art = ART(A, AT, y, x0, mu, niter, bpos)


## DISPLAY
wndImg = [0, 0.03]
wndPrj = [0, 6]

plt.subplot(241)
plt.imshow(x, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('Ground truth')

plt.subplot(242)
plt.imshow(x_full, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('full-dose')

mse_x_low=mse(x,x_low)
psnr_x_low=psnr(x,x_low)
ssim_x_low=ssim(x,x_low)

plt.subplot(243)
plt.imshow(x_low, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('low-dose\nMSE: %.4f\nPSNR: %.4f\nSSIM: %.4f' % (mse_x_low, psnr_x_low, ssim_x_low))

mse_x_art=mse(x,x_art)
psnr_x_art=psnr(x,x_art)
ssim_x_art=ssim(x,x_art)

plt.subplot(244)
plt.imshow(x_art, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('ART\nMSE: %.4f\nPSNR: %.4f\nSSIM: %.4f' % (mse_x_art, psnr_x_art, ssim_x_art))

plt.subplot(246)
plt.imshow(p, cmap='gray', vmin=wndPrj[0], vmax=wndPrj[1])
plt.title('full-dose\n(VIEW: %d)' % VIEW)
plt.xlabel('View')
plt.ylabel('Detector')

plt.subplot(247)
plt.imshow(y, cmap='gray', vmin=wndPrj[0], vmax=wndPrj[1])
plt.title('low-dose\n(VIEW: %d)' % VIEW)
plt.xlabel('View')
plt.ylabel('Detector')

plt.subplot(248)
plt.imshow(y - p, cmap='gray')
plt.title('full-dose - low-dose\n(Poisson noise)')
plt.xlabel('View')
plt.ylabel('Detector')

plt.show()


## Ahora modificaciones sorbe la demo

Basicamente voy a sacar casi todos los plot y dejo nomas el final de la reconstrucción. Por otro lado voy a agregar el calculo del error con la funcion de arriba. Y voy a ahcer esto para varios valores de i0

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 14 09:06:26 2022

@author: matog
"""
## REFERENCE
# https://en.wikipedia.org/wiki/Algebraic_reconstruction_technique

## ART Equation
# x^(k+1) = x^k + lambda * AT(b - A(x))/ATA

import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import radon, iradon
from scipy.io import loadmat
from scipy.stats import poisson
#from skimage.measure import compare_mse as mse
#from skimage.measure import compare_psnr as psnr
#from skimage.measure import compare_ssim as ssim
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import time


def demo(N=512, ANG=180, VIEW=360, i0=1e2, niter = 1e2):
  ## SYSTEM SETTING
  THETA = np.linspace(0, ANG, VIEW + 1)
  THETA = THETA[:-1]
  
  tiempo_bp = 0 #Guarda el tiempo que tarda retroproyectar
  tiempo_fp = 0 #Guarda el tiempo que tarda art

  A = lambda x: radon(x, THETA, circle=False).astype(np.float32)
  AT = lambda y: iradon(y, THETA, circle=False, filter_name=None, output_size=N).astype(np.float32)/(np.pi/(2*len(THETA)))
  AINV = lambda y: iradon(y, THETA, circle=False, filter_name='ramp',output_size=N).astype(np.float32)

  ## DATA GENERATION
  x = loadmat('XCAT512.mat')['XCAT512']

  from skimage.transform import resize
  x = resize(x, (N, N), order=3, preserve_range=True) # Use bicubic interpolation (order=3)


  p = A(x)
  x_full = AINV(p)


  ## LOW-DOSE SINOGRAM GENERATION
  pn = np.exp(-p)
  pn = i0*pn
  pn = poisson.rvs(pn)
  pn[pn <1] = 1
  pn = -np.log(pn/i0)
  pn[pn < 0] = 0

  y = pn

  ## Algebraic Reconstruction Technique (ART) INITIALIZATION
  
  #Aca tomamos tiempo para retroproyectar
  start = time.time()
  x_low = AINV(y)
  end = time.time()
  tiempo_bp = end - start
  
  x0 = np.zeros_like(x)
  mu = 1e0
  bpos = True

  #Tomo el tiempo para ART
  start = time.time()
  x_art = ART(A, AT, y, x0, mu, niter, bpos)
  end = time.time()
  tiempo_fp = end - start

  ## DISPLAY
  wndImg = [0, 0.03]
  wndPrj = [0, 6]


  #ssim_x_low=ssim(x,x_low)

  plt.subplot(241)
  plt.imshow(x, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
  plt.axis('off')
  plt.axis('image')
  plt.title('Ground truth')

  plt.subplot(242)
  plt.imshow(x_full, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
  plt.axis('off')
  plt.axis('image')
  plt.title('full-dose')

  plt.subplot(243)
  plt.imshow(x_low, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
  plt.axis('off')
  plt.axis('image')
  plt.title('low-dose')


  #ssim_x_art=ssim(x,x_art)

  plt.subplot(244)
  plt.imshow(x_art, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
  plt.axis('off')
  plt.axis('image')
  plt.title('ART')

  #Meto un suptitle con los parametros
  plt.suptitle(f'Angulos = {ANG}, Sensores = {N}, Intensidad = {i0}')
  plt.show()

  return x_art, x_low, tiempo_bp, tiempo_fp



## Calculo del error en función del nro de angulos

In [ ]:
#Config default, Sensores 512, i0 1e2 de 0 a 360°
x = loadmat('XCAT512.mat')['XCAT512']
angulos = np.linspace(0,400,10)

errores_ang = []
i = 0
for angulo in angulos:
  x_art, x_low = demo(ANG=angulo)
  error = (calculate_rmse(x_art, x), calculate_rmse(x_art, x_low))
  errores_ang.append(error)
  print(f'Angulo {angulo}. Error art-file = {error[0]}. Error art-inversa = {error[1]}')

  i+=1

print(errores_ang)


## Aca lo hago para el numero de sensores

In [ ]:
from skimage.transform import resize

sensores = np.linspace(256,512,5)
x = loadmat('XCAT512.mat')['XCAT512']


errores_sensores = []

for sensor in sensores:
  img = resize(x, (int(sensor), int(sensor)), order=3, preserve_range=True)
  x_art, x_low = demo(N=int(sensor), niter=20)
  error = (calculate_rmse(x_art, img), calculate_rmse(x_low, img))
  errores_sensores.append(error)

print(errores_sensores)

In [11]:
for i, error in enumerate(errores_sensores):
    print(f'{sensores[i]}\t{error[0]}\t{error[1]}')

## Ahora lo mismo pero para el i0

In [ ]:
intensidades = np.linspace(1e0,1e3,5)
x = loadmat('XCAT512.mat')['XCAT512']


errores_intesidades = []

for i0 in intensidades:
  x_art, x_low = demo(i0=i0)
  error = (calculate_rmse(x_art, x), calculate_rmse(x_low, x))
  errores_intesidades.append(error)

print(errores_intesidades)

# Generalización de TODO:

In [ ]:
import csv
import numpy as np
from scipy.io import loadmat
from skimage.transform import resize

# Cargar datos
x = loadmat('XCAT512.mat')['XCAT512']
angulos = np.linspace(0, 400, 10)
sensores = np.linspace(256, 512, 5)
intensidades = np.linspace(1e0, 1e3, 5)


def errores_angulos(csvfile, x=x, angulos=angulos):
    writer = csv.writer(csvfile)
    writer.writerow(['Angulo', 'Error art-orig', 'Error art-low', 'Tiempo BP', 'Tiempo ART'])  # Encabezado
    
    for angulo in angulos:
        x_art, x_low, tiempo_bp, tiempo_art = demo(ANG=angulo)
        error = (calculate_rmse(x_art, x), calculate_rmse(x_art, x_low))
        print(f'Angulo {angulo}. Error art-orig = {error[0]}. Error art-low = {error[1]}')
        writer.writerow([angulo, error[0], error[1], tiempo_bp, tiempo_art])


def errores_sensores(csvfile, x=x, sensores=sensores):
    writer = csv.writer(csvfile)
    writer.writerow(['Sensores', 'Error art-orig', 'Error art-low', 'Tiempo BP', 'Tiempo ART'])
    
    for sensor in sensores:
        img = resize(x, (int(sensor), int(sensor)), order=3, preserve_range=True)
        x_art, x_low, t_bp, t_art = demo(N=int(sensor), niter=20)
        error = (calculate_rmse(x_art, img), calculate_rmse(x_low, img))
        writer.writerow([sensor, error[0], error[1], t_bp, t_art])


def errores_intensidades(csvfile, x=x, intensidades=intensidades):
    writer = csv.writer(csvfile)
    writer.writerow(['Intensidad', 'Error art-orig', 'Error art-low', 'Tiempo BP', 'Tiempo ART'])
    
    for i0 in intensidades:
        x_art, x_low, t_bp, t_art = demo(i0=i0)
        error = (calculate_rmse(x_art, x), calculate_rmse(x_low, x))
        writer.writerow([i0, error[0], error[1], t_bp, t_art])


# Guardar archivos en CSV
with open('errores_angulos.csv', 'w', newline='') as f:
    #Vamos agregar una linea con param iniciales
    
    errores_angulos(f)

with open('errores_sensores.csv', 'w', newline='') as f:
    errores_sensores(f)

with open('errores_intensidades.csv', 'w', newline='') as f:
    errores_intensidades(f)
